# <font color="purple">Adversarial Training of Residual Network</font>

Definition of WideResNet and Architecture:

##  <center>The Result of Residual Network </center>

***Road Map***
* Data Preprocessing
* Model Cross Validation Results
* Evaluate the GridSearchCV Results
* Model Training and Learning Curves
* Model Adversarial Training Approach.


**Libraries**

In [ ]:
import sys
sys.path.insert(1,'/home/sefika/AE_Parseval_Network/src/cleverhans/future/tf2/attacks')
import cleverhans


In [2]:
!pip install -qq -e git+http://github.com/tensorflow/cleverhans.git#egg=cleverhans
import sys
sys.path.append('/content/src/cleverhans')
import cleverhans

     |████████████████████████████████| 163kB 4.7MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 


In [3]:
import tensorflow as tf
from cleverhans.future.tf2.attacks import fast_gradient_method
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import  KFold
import gzip
import pickle
import numpy as np
from parsevalnet import ParsevalNetwork
import warnings
warnings.filterwarnings("ignore")

print("\nTensorflow Version: " + tf.__version__)
# utility functions
from preprocessing import preprocessing_data
# Define configuration parameters
from _utility import lrate
from training import train
from adversarial_training import  AdversarialTraining


Tensorflow Version: 2.3.0


## <font color="green"> Data Preprocessing </font>


* Read Data from File

In [4]:
def read_data():
    with open("data.pz", 'rb') as file_:
        with gzip.GzipFile(fileobj=file_) as gzf:
            data = pickle.load(gzf, encoding='latin1', fix_imports=True)
    return data
data = read_data()

* Call data preprocessing function

In [5]:
X, y = preprocessing_data(data)
X_train, X_test, Y_train, y_test = train_test_split(X, y, test_size = 0.1)
x_train, X_val, y_train, y_val = train_test_split(X_train, Y_train, test_size = 0.1)

## <font color="green"> Utilize Functions </font>

* Flipping the image using data augmentation technique

In [6]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=10,
                               width_shift_range=5./32,
                               height_shift_range=5./32,)

### <font color = "green">Some Parameters Regarding Adversarial Examples</font>

In [7]:
# predefined epsilon values
epsilon_list = [0.003,0.005,0.01,0.02]

## <font color="purple"> 1.) Baseline of the Model</font>

In [8]:
EPOCHS = 50
BS = 64
init = (32, 32,1)
sgd = SGD(lr=0.1, momentum=0.9)
parameter = {'epochs': EPOCHS, 'batch_size': BS, 'optimizer': sgd}

In [9]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
callbacks_list = [lrate, es]
parseval = ParsevalNetwork(init,0.0001, 0.9, nb_classes=4, N=2, k=1, dropout=0.0)

In [10]:
result_df = train(parseval, X_train, Y_train, X_test, y_test, EPOCHS, BS, sgd, generator, callbacks_list, epsilon_list, model_name="Parseval")

18/18 [==============================] - 0s 4ms/step - loss: 1.1346 - acc: 0.5812
epsilon: 0.01 and test evaluation : 1.134591817855835, 0.5811518430709839
SNR: 39.777822494506836
18/18 [==============================] - 0s 4ms/step - loss: 1.5816 - acc: 0.4328
epsilon: 0.02 and test evaluation : 1.5815844535827637, 0.4328097701072693
SNR: 33.757221698760986


In [11]:
result_df["clean_mean"] = np.sum(result_df['acc_clean'])/10.0
result_df["0.003_mean"] = np.sum(result_df['0.003_acc'])/10.0
result_df["0.005_mean"] = np.sum(result_df['0.005_acc'])/10.0
result_df["0.02_mean"] = np.sum(result_df['0.02_acc'])/10.0
result_df["0.01_mean"] = np.sum(result_df['0.01_acc'])/10.0

In [12]:
result_df.head(1)

,loss_clean,acc_clean,0.003_loss,0.003_acc,0.005_loss,0.005_acc,0.02_acc,0.02_loss,0.01_acc,0.01_clean,0.01_loss,clean_mean,0.003_mean,0.005_mean,0.02_mean,0.01_mean
0,0.748837,0.727749,0.841311,0.678883,0.906478,0.652705,0.577661,1.077555,0.458988,NaN,0.458988,0.735777,0.688133,0.658988,0.57644,0.429319


## <font color="purple">2.) Adversarial Training on Baseline Model</font>

In [13]:
adversarial_training =  AdversarialTraining(parameter)
result_adv_df = adversarial_training.train(parseval, X_train, Y_train, X_test, y_test, epsilon_list, callbacks_list,model_name="parseval")

Parseval  Network-16-1 created.
Finished compiling
[0.003, 0.005, 0.01, 0.02]
Epoch 1/50
72/72 [==============================] - 2s 27ms/step - loss: 1.3645 - acc: 0.3348 - val_loss: 1.2921 - val_acc: 0.4155
Epoch 2/50
72/72 [==============================] - 2s 22ms/step - loss: 1.2777 - acc: 0.3930 - val_loss: 1.2139 - val_acc: 0.4757
Epoch 3/50
72/72 [==============================] - 2s 22ms/step - loss: 1.2305 - acc: 0.4678 - val_loss: 1.2233 - val_acc: 0.4680
Epoch 4/50
72/72 [==============================] - 2s 22ms/step - loss: 1.1799 - acc: 0.5079 - val_loss: 1.1851 - val_acc: 0.4971
Epoch 5/50
72/72 [==============================] - 2s 22ms/step - loss: 1.1339 - acc: 0.5311 - val_loss: 1.1567 - val_acc: 0.5087
Epoch 6/50
72/72 [==============================] - 2s 22ms/step - loss: 1.0786 - acc: 0.5632 - val_loss: 1.2526 - val_acc: 0.4854
Epoch 7/50
72/72 [==============================] - 2s 22ms/step - loss: 1.0398 - acc: 0.5840 - val_loss: 0.9518 - val_acc: 0.5942
Epoch

In [14]:
result_adv_df

,loss_clean,acc_clean,0.003_loss,0.003_acc,0.005_loss,0.005_acc,0.02_acc,0.02_loss,0.01_acc,0.01_loss
0,0.730672,0.746946,0.867045,0.698080,0.967964,0.656195,0.547993,1.242121,0.431065,1.803481
1,0.753106,0.729494,0.860288,0.685864,0.936263,0.645724,0.572426,1.138414,0.450262,1.558927
2,0.720366,0.738220,0.851248,0.692845,0.946014,0.657941,0.579407,1.203322,0.429319,1.747294
3,0.727960,0.743455,0.828039,0.710297,0.899946,0.684119,0.610820,1.094225,0.465969,1.505179
4,0.827427,0.671902,0.964707,0.616056,1.062576,0.575916,0.492147,1.318801,0.383944,1.824190
5,0.717536,0.732984,0.827040,0.684119,0.905636,0.659686,0.588133,1.116872,0.436300,1.561108
6,0.763113,0.712042,0.867792,0.671902,0.942239,0.631763,0.547993,1.140204,0.424084,1.548451
7,0.722233,0.762653,0.838029,0.691099,0.921562,0.664921,0.570681,1.146798,0.439791,1.611803
8,0.751309,0.719023,0.843336,0.694590,0.908345,0.664921,0.603839,1.079059,0.462478,1.436870
9,0.710245,0.732984,0.826461,0.694590,0.910326,0.659686,0.570681,1.134904,0.439791,1.595775


### <font color="purple"> Results of Adversarial Training</font>

In [15]:
result_adv_df["clean_mean"] = np.sum(result_adv_df['acc_clean'])/10.0
result_adv_df["0.003_mean"] = np.sum(result_adv_df['0.003_acc'])/10.0
result_adv_df["0.005_mean"] = np.sum(result_adv_df['0.005_acc'])/10.0
result_adv_df["0.02_mean"] = np.sum(result_adv_df['0.02_acc'])/10.0
result_adv_df["0.01_mean"] = np.sum(result_adv_df['0.01_acc'])/10.0

In [16]:
column = ["clean_mean","0.003_mean","0.005_mean","0.02_mean","0.01_mean"]
result_adv_df[column].head(1)

,clean_mean,0.003_mean,0.005_mean,0.02_mean,0.01_mean
0,0.72897,0.683944,0.650087,0.568412,0.4363


# <font color="blue">Compare Non-Adversarial Training with Adversarial Training</font>

In [17]:
result_adv_df[column].head(1)

,clean_mean,0.003_mean,0.005_mean,0.02_mean,0.01_mean
0,0.72897,0.683944,0.650087,0.568412,0.4363


In [18]:
result_df[column].head(1)

,clean_mean,0.003_mean,0.005_mean,0.02_mean,0.01_mean
0,0.735777,0.688133,0.658988,0.57644,0.429319


# <font color="blue">Conclusion</font>